In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs
from geo_tool import Point_Cloud
from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.point_clouds.latent_gan import LatentGAN
from tf_lab.point_clouds.generators_discriminators import latent_code_discriminator_two_layers, \
latent_code_generator_two_layers, latent_code_discriminator, latent_code_generator
from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats
from tf_lab.point_clouds.convenience import get_latent_codes
from tf_lab.neural_net import MODEL_SAVER_ID
from tf_lab.point_clouds.raw_gan_gp import RawGAN_GP
from tf_lab.point_clouds.generators_discriminators import latent_code_generator

In [4]:
from general_tools.in_out.basics import pickle_data

In [5]:
n_pc_points = 2048
random_seed = 42
all_classes = ['table', 'chair', 'sofa']
#                'car', 'airplane']
# , 'rifle']
top_data_dir = '/orions4-zfs/projects/optas/DATA/'

In [6]:
# Load Synthetic Data.
all_syn_data = []
all_syn_labels = []
for obj_label, obj_class in enumerate(all_classes):
    gmm_syn_data = osp.join(top_data_dir, 'OUT/iclr/synthetic_samples/gmm/gmm_emd_'+
                             obj_class, 'bneck_128_full_32_gaussians.npz')
    
    syn_data = np.load(gmm_syn_data)
    syn_data = syn_data[syn_data.keys()[0]]

    all_syn_data.append(syn_data)
    all_syn_labels.append(np.ones(len(syn_data)) * obj_label)

all_syn_data = np.vstack(all_syn_data)
all_syn_labels = np.hstack(all_syn_labels)

syn_data = PointCloudDataSet(all_syn_data, labels=all_syn_labels)

In [7]:
# # LOAD ORIGINAL DATA.
all_original_data = []
for obj_label, obj_class in enumerate(all_classes):
    syn_id = snc_category_to_synth_id()[obj_class]
    in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)
    all_original_data.append(in_data)

852 pclouds were loaded. They belong in 1 shape-classes.
7232 pclouds were loaded. They belong in 1 shape-classes.
425 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
318 pclouds were loaded. They belong in 1 shape-classes.
2697 pclouds were loaded. They belong in 1 shape-classes.
158 pclouds were loaded. They belong in 1 shape-classes.


In [8]:
def extract_split_from_original_data(all_original_data, split):
    original_split_data = []
    original_split_labels = []
    for obj_label, obj_data in enumerate(all_original_data):
        original_split_data.append(obj_data[split].point_clouds)
        original_split_labels.append(np.ones(obj_data[split].num_examples) * obj_label)

    original_split_data = np.vstack(original_split_data)
    original_split_labels = np.hstack(original_split_labels)
    pure_split_data = PointCloudDataSet(original_split_data, labels=original_split_labels,
                                       init_shuffle=False)
    return pure_split_data
    

In [9]:
pure_train_data = extract_split_from_original_data(all_original_data, 'train')
mixed_train_data = PointCloudDataSet(np.vstack([pure_train_data.point_clouds, syn_data.point_clouds]),
                                     labels=np.hstack([pure_train_data.labels, syn_data.labels]))

In [10]:
n_classes = len(all_classes)
out_dir = 'OUT/icml/augmented_training_data/' + str(n_classes) + '_sn_classes'
create_dir(osp.join(top_data_dir, out_dir))

pickle_data(osp.join(top_data_dir, out_dir, 'syn_data'), syn_data)

pickle_data(osp.join(top_data_dir, out_dir, 'mixed_train_data'), mixed_train_data)
pickle_data(osp.join(top_data_dir, out_dir, 'pure_train_data'), pure_train_data)

pure_test_data = extract_split_from_original_data(all_original_data, 'test')
pickle_data(osp.join(top_data_dir, out_dir, 'pure_test_data'), pure_test_data)

pure_val_data = extract_split_from_original_data(all_original_data, 'val')
pickle_data(osp.join(top_data_dir, out_dir, 'pure_val_data'), pure_val_data)

In [42]:
# LOOK LATER FOR GIVING AN ANSWER about the CLF of the multi-class produced data.

In [ ]:
# class_name = 'achlioptas_5_snc'

In [ ]:

in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)
train_data = in_data['train']

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/achlioptas_snc_5_medium_size_splits/train.txt')
val_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/achlioptas_snc_5_medium_size_splits/val.txt')
splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)

tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

val_files = splitter.load_splits(val_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(val_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/in_out.py:38: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


10000 pclouds were loaded. They belong in 5 shape-classes.
500 pclouds were loaded. They belong in 5 shape-classes.


In [10]:
# evaluate Point-net here.
# syn_data = osp.join(top_data_dir, 'OUT/iclr/synthetic_samples/l_w_gan/l_w_gan_achlioptas_5_snc_mlp_with_split_1pc_usampled_emd_bneck_128/epoch_2000.npz')
# syn_data = np.load(syn_data)
# syn_data = syn_data[syn_data.keys()[0]]